<a href="https://colab.research.google.com/github/harithaharry001/CovidDataset/blob/master/DenseNet_121.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Dense
from tensorflow.keras.layers import AvgPool2D, GlobalAveragePooling2D, MaxPool2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import ReLU, concatenate
import tensorflow.keras.backend as K

In [4]:
# Creating Densenet121
def densenet(input_shape, n_classes, filters = 32):
    
    #batch norm + relu + conv
    def bn_rl_conv(x,filters,kernel=1,strides=1):
        
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(filters, kernel, strides=strides,padding = 'same')(x)
        return x
    
    def dense_block(x, repetition):
        
        for _ in range(repetition):
            y = bn_rl_conv(x, 4*filters)
            y = bn_rl_conv(y, filters, 3)
            x = concatenate([y,x])
        return x
        
    def transition_layer(x):
        
        x = bn_rl_conv(x, K.int_shape(x)[-1] //2 )
        x = AvgPool2D(2, strides = 2, padding = 'same')(x)
        return x
    
    input = Input (input_shape)
    x = Conv2D(64, 7, strides = 2, padding = 'same')(input)
    x = MaxPool2D(3, strides = 2, padding = 'same')(x)
    
    for repetition in [6,12,24,16]:
        
        d = dense_block(x, repetition)
        x = transition_layer(d)
    x = GlobalAveragePooling2D()(d)
    output = Dense(n_classes, activation = 'softmax')(x)
    
    model = Model(input, output)
    return model


In [20]:
input_shape = 224, 224, 3
n_classes = 2
model = densenet(input_shape,n_classes)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_121 (Conv2D)             (None, 112, 112, 64) 9472        input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 56, 56, 64)   0           conv2d_121[0][0]                 
__________________________________________________________________________________________________
batch_normalization_120 (BatchN (None, 56, 56, 64)   256         max_pooling2d_1[0][0]            
____________________________________________________________________________________________

In [8]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from os import listdir
from matplotlib import image
from skimage.transform import resize


def load_data(path):
    
    train_files = []
    for foldername1 in listdir(path):
        filepath1 = path  + "/" + foldername1
        for filename1 in listdir(filepath1):
            train_files.append(filepath1 + "/" + filename1)
            
    # Original Dimensions
    image_width = 224
    image_height = 224
    channels = 3
    
    loaded_images = np.ndarray(shape=(len(train_files), image_height, image_width, channels),dtype=np.float32)
    print(type(loaded_images))
    loaded_class = []
    i = 0
    for foldername in listdir(path):
        filepath = path  + "/" + foldername
        print("Folder : ",filepath)
        for filename in listdir(filepath):
            # load image
            img_data = image.imread(filepath + "/" + filename)
            
            # store loaded image
            img_data = resize(img_data, (224, 224, 3))
            loaded_images[i] = img_data
            loaded_class.append(foldername)
            i = i + 1
            #print('> loaded %s %s' % (filename, img_data.shape))
        print('Loaded: ',i , ' images from ',filepath)
            
    return loaded_images,loaded_class


In [9]:
path = r"/content/drive/MyDrive/Dataset"
x,y = load_data(path)
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.3)

<class 'numpy.ndarray'>
Folder :  /content/drive/MyDrive/Dataset/Covid
Loaded:  478  images from  /content/drive/MyDrive/Dataset/Covid
Folder :  /content/drive/MyDrive/Dataset/Normal
Loaded:  2358  images from  /content/drive/MyDrive/Dataset/Normal


In [12]:
print(train_x.shape)
print(test_x.shape)

(1650, 224, 224, 3)
(708, 224, 224, 3)


In [13]:
import pandas as pd
from keras.utils import to_categorical
y_train_values, unique = pd.factorize(train_y)
print('y_train ', y_train_values, unique)


y_test_values, unique = pd.factorize(test_y)
print('y_test ', y_test_values, unique)

y_train_one_hot = to_categorical(y_train_values)
y_test_one_hot = to_categorical(y_test_values)


y_train  [0 0 0 ... 0 0 0] ['Normal' 'Covid']
y_test  [0 1 1 1 1 1 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1
 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 0
 0 1 0 1 0 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1
 1 0 1 1 1 1 1 0 0 1 1 0 1 1 1 0 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 0
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 0 1 1
 1 0 1 1 1 1 1 0 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 1 0 1 1 1 1 1 0 1 0 1 1 1 0
 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 0 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 0 1 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 0 1
 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0

In [14]:
x_train_normalization = train_x / 255.0
x_test_normalization = test_x / 255.0

In [18]:
from sklearn.utils import shuffle


x_shuffled_default, y_shuffled_default = shuffle(x_train_normalization, y_train_one_hot)

In [22]:
model.compile(
loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(x_shuffled_default, y_shuffled_default, epochs = 30, batch_size = 32)

Epoch 1/30
52/52 [==============================] - 1160s 22s/step - loss: 0.6936 - accuracy: 0.8154
Epoch 2/30
52/52 [==============================] - 1177s 23s/step - loss: 0.3477 - accuracy: 0.8344
Epoch 3/30
52/52 [==============================] - 1231s 24s/step - loss: 0.2700 - accuracy: 0.8674
Epoch 4/30
52/52 [==============================] - 1236s 24s/step - loss: 0.2896 - accuracy: 0.8644
Epoch 5/30
52/52 [==============================] - 1207s 23s/step - loss: 0.2710 - accuracy: 0.8511
Epoch 6/30
52/52 [==============================] - 1200s 23s/step - loss: 0.2229 - accuracy: 0.8880
Epoch 7/30
52/52 [==============================] - 1197s 23s/step - loss: 0.2247 - accuracy: 0.8886
Epoch 8/30
52/52 [==============================] - 1198s 23s/step - loss: 0.2125 - accuracy: 0.8850
Epoch 9/30
52/52 [==============================] - 1229s 24s/step - loss: 0.2240 - accuracy: 0.8746
Epoch 10/30
52/52 [==============================] - 1253s 24s/step - loss: 0.2184 - accura